In [1]:
import requests as http
from io import StringIO

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from av_lib import *

In [2]:
api_key = None
with open('api-key.txt') as file:
    api_key = file.read()

In [3]:
url = daily_adjusted_query(api_key, symbol='IBM', output_size='full')
req = http.get(url)

In [4]:
data = None
if req.status_code == 200:
    data = str(req.content, 'utf-8')
data = StringIO(data)
data = pd.read_csv(data)
data

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-05,147.94,148.9300,146.80,147.41,147.410000,2784767,0.0,1.0
1,2022-12-02,148.13,149.1600,147.73,148.67,148.670000,2899995,0.0,1.0
2,2022-12-01,149.98,150.0100,147.34,149.16,149.160000,4495924,0.0,1.0
3,2022-11-30,146.19,149.6400,145.67,148.90,148.900000,6377582,0.0,1.0
4,2022-11-29,145.91,147.1668,145.70,146.49,146.490000,2754744,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5807,1999-11-05,92.75,92.9400,90.19,90.25,49.728782,13737600,0.0,1.0
5808,1999-11-04,94.44,94.4400,90.00,91.56,50.450607,16697600,0.0,1.0
5809,1999-11-03,95.87,95.9400,93.50,94.37,51.998949,10369100,0.0,1.0
5810,1999-11-02,96.75,96.8100,93.69,94.81,52.241394,11105400,0.0,1.0


In [5]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-05,147.94,148.9300,146.80,147.41,147.41,2784767,0.0,1.0
1,2022-12-02,148.13,149.1600,147.73,148.67,148.67,2899995,0.0,1.0
2,2022-12-01,149.98,150.0100,147.34,149.16,149.16,4495924,0.0,1.0
3,2022-11-30,146.19,149.6400,145.67,148.90,148.90,6377582,0.0,1.0
4,2022-11-29,145.91,147.1668,145.70,146.49,146.49,2754744,0.0,1.0


In [7]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Candlestick(
            x = data['timestamp'],
            open = data['open'],
            high = data['high'],
            low = data['low'],
            close = data['close'],
            name = 'IBM'
        )
    ],
    layout_title = 'IBM Daily Price'
)

fig.update_layout(
    autosize=True,
    height = 700,
    margin = {
        'l': 50,
        'r': 50,
        'b': 50,
        't': 100,
        'pad': 4 
    }
)

fig.show()